In [4]:
import pandas as pd
import numpy as np

In [5]:
YEAR = 2021

In [6]:
df = pd.read_csv(f'../dataset/summary_{YEAR}.csv', encoding='utf-8-sig')

def read_file(year):
    PATH = f'../dataset/imbank_customer_data/iMBANK_CUSTOMER_DATA_{year}(K-DigitalTraining).csv'
    df = pd.read_csv(PATH, encoding='euc-kr', usecols=['기준년월', '고객ID', '성별', '고객등급', '자택_시도', '수신_요구불예금', '수신_거치식예금', '수신_적립식예금', '대출금액'])
    df = df.sort_values(by='기준년월', ascending=True).reset_index(drop=True)
    return df

M_CS = read_file(YEAR)

# 고객 id 
filtered_data = df[df['연령대'].isin(['20대', '30대'])]['고객ID'].unique()

# 20, 30대의 고객 ID만 뽑음
M_filtered_data = M_CS[M_CS['고객ID'].isin(filtered_data)]

# 수신잔고, sum(요구불 ~ 적립식)
M_filtered_data['수신잔고'] = M_filtered_data.groupby(['기준년월','고객ID'])[['수신_요구불예금', '수신_거치식예금', '수신_적립식예금']].transform('sum').sum(axis=1)

# 지역구분 대구, 경상북도 => 1, 나머지 => 0
M_filtered_data['지역구분'] = np.where(M_filtered_data['자택_시도'].isin(['대구광역시', '경상북도']), 1, 0)                                                                

# 수신대비대출비율
M_filtered_data['수신대비대출비율'] = np.where(M_filtered_data['수신잔고'] != 0, M_filtered_data['대출금액'] / M_filtered_data['수신잔고'], 0)

result = M_filtered_data.merge(df[['고객ID', '연령대', '이탈여부']], on='고객ID', how='left')
result

C:\Users\campus3S024\AppData\Local\Temp\ipykernel_11980\232365773.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M_filtered_data['수신잔고'] = M_filtered_data.groupby(['기준년월','고객ID'])[['수신_요구불예금', '수신_거치식예금', '수신_적립식예금']].transform('sum').sum(axis=1)
C:\Users\campus3S024\AppData\Local\Temp\ipykernel_11980\232365773.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M_filtered_data['지역구분'] = np.where(M_filtered_data['자택_시도'].isin(['대구광역시', '경상북도']), 1, 0)
C:\Users\campus3S024\AppData\Local\Temp\ipykernel

,기준년월,고객ID,성별,고객등급,자택_시도,수신_요구불예금,수신_거치식예금,수신_적립식예금,대출금액,수신잔고,지역구분,수신대비대출비율,연령대,이탈여부
0,202101,3032273566,남,일반등급,서울특별시,0,0,0,0,0,0,0.000000,30대,유지
1,202101,3032766944,남,일반등급,대전광역시,100000,0,0,100000,100000,0,1.000000,20대,이탈
2,202101,3032863476,남,일반등급,경기도,0,0,0,0,0,0,0.000000,30대,유지
3,202101,3032782859,남,일반등급,서울특별시,0,0,0,0,0,0,0.000000,30대,이탈
4,202101,3032966180,여,일반등급,대구광역시,0,0,0,0,0,1,0.000000,20대,유지
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2786412,202112,9996689486,여,일반등급,대구광역시,5800000,0,0,0,5800000,1,0.000000,30대,유지
2786413,202112,9996720726,남,일반등급,대구광역시,1200000,0,0,0,1200000,1,0.000000,30대,이탈
2786414,202112,9996732244,남,일반등급,경상북도,7200000,0,0,0,7200000,1,0.000000,30대,유지
2786415,202112,9996756045,남,일반등급,경상북도,300000,0,0,0,300000,1,0.000000,30대,유지


In [ ]:
result = M_filtered_data.merge(df[['고객ID', '연령대', '이탈여부']], on='고객ID', how='left')

In [8]:
result.to_csv(f'../dataset/imbank_customer_data/customer_newCols_{YEAR}.csv', encoding='euc-kr')

In [ ]:
def read_file1(year):
    PATH = f'../dataset/imbank_card_data/iMBANK_CARD_DATA_{year}(K-DigitalTraining).csv'
    df = pd.read_csv(PATH, encoding='euc-kr', usecols=['거래년월','고객ID','승인건수','승인금액'])
    df = df.sort_values(by='거래년월', ascending=True).reset_index(drop=True)
    # 승인건수를 숫자로 변환 (필요한 경우 오류 처리)
    
    # 승인건수를 범주화
    df.loc[df['승인건수'] == '5번이하', '승인건수'] = 1
    df.loc[df['승인건수'] == '5번초과 10번이하', '승인건수'] = 2
    df.loc[df['승인건수'] == '10번초과 15번이하', '승인건수'] = 3
    df.loc[df['승인건수'] == '15번초과 20번이하', '승인건수'] = 4
    df.loc[df['승인건수'] == '20번 초과', '승인건수'] = 5

    card_date =df.groupby(['거래년월', '고객ID'])
    p = card_date['승인건수'].sum() / card_date['승인건수'].count()
    p = p.reset_index()

    

    new_card = pd.merge(new_card, p, on=['거래년월', '고객ID'], how='left')
    new_card.drop(['승인건수_x'], axis = 1, inplace=True)
    _sum = new_card.groupby(['거래년월', '고객ID'])['승인금액'].transform('sum')

    new_card = pd.merge(new_card, _sum, on=['거래년월', '고객ID'], how='left')
    new_card.columns = ['거래년월', '고객ID', '이탈여부', '승인건수', '월별승인금액합계']
    new_card.to_csv(f"../dataset/imbank_card_data/samp_card_{year}.csv", encoding='euc-kr', index=False)
    return df

C = read_file1(YEAR)

UnboundLocalError: cannot access local variable 'new_card' where it is not associated with a value

In [ ]:
# 이전 고객ID, 연령, 이탈여부 summary파일 불러오기
dist = pd.read_csv('../dataset/summary_{YEAR}.csv')
dist.head(3)


FileNotFoundError: [Errno 2] No such file or directory: '../dataset/summary_{YEAR}.csv'

In [ ]:

# inner join(card2022 <- summary 파일)
cs_merge = C.merge(dist, on = '고객ID', how = 'inner')
cs_merge.head(15)